In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-clickbait-dataset/train1.csv
/kaggle/input/news-clickbait-dataset/train2.csv


In [27]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import re

In [28]:
train1_data = pd.read_csv("../input/news-clickbait-dataset/train1.csv")
Headlines = train1_data["headline"]
Classification = train1_data["clickbait"]

In [29]:
def clean_text(text):
  text = re.sub(r',', '', text)
  text = re.sub(r'\'', '',  text)
  text = re.sub(r'\"', '', text)
  text = re.sub(r'\(', '', text)
  text = re.sub(r'\)', '', text)
  text = re.sub(r'\n', '', text)
  text = re.sub(r'“', '', text)
  text = re.sub(r'”', '', text)
  text = re.sub(r'’', '', text)
  text = re.sub(r'\.', '', text)
  text = re.sub(r';', '', text)
  text = re.sub(r':', '', text)
  text = re.sub(r'\-', '', text)

  return text


In [51]:
clickbait_sentences = []

for i,headline in enumerate(Headlines):
    if Classification[i] == 1:
        clickbait_sentences.append(clean_text(headline))
        
clickbait_sentences = [text.lower() for text in clickbait_sentences]

In [50]:
# Preprocess and tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clickbait_sentences)
indexed_sentences = tokenizer.texts_to_sequences(clickbait_sentences)
vocab_size = len(tokenizer.word_index) + 1

In [32]:
# Create n-gram sequences from the sentences
input_seq = []
for sentence in indexed_sentences:
    for i in range(1, len(sentence)):
        n_gram_seq = sentence[:i+1]
        input_seq.append(n_gram_seq)

In [33]:
# Pad the sequences
max_seq_length = max(len(x) for x in input_seq)
input_seq = pad_sequences(input_seq, maxlen=max_seq_length, padding='pre')

# Split the input sequences into input (xs) and label (labels)
xs = input_seq[:, :-1]
labels = input_seq[:, -1]

# One-hot encode the labels
ys = tf.keras.utils.to_categorical(labels, num_classes=vocab_size)

In [34]:
# Define the model architecture
input_size = max_seq_length - 1
embedding_size = 124
lstm_units = 520
bidirectional_lstm_units = 340
dense_units = 1024

input_layer = Input(shape=(input_size,))
embedding_layer = Embedding(vocab_size, embedding_size)(input_layer)
dropout_layer = Dropout(0.2)(embedding_layer)
lstm_layer = LSTM(lstm_units, return_sequences=True)(dropout_layer)
bidirectional_lstm_layer = Bidirectional(LSTM(bidirectional_lstm_units, return_sequences=True))(lstm_layer)
pooling_layer = GlobalMaxPooling1D()(bidirectional_lstm_layer)
dense_layer1 = Dense(dense_units, activation='relu')(pooling_layer)
output_layer = Dense(vocab_size, activation='softmax')(dense_layer1)

model = Model(input_layer, output_layer)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(xs, ys, epochs=100)

Epoch 1/100
4471/4471 [==============================] - 119s 24ms/step - loss: 6.8691 - accuracy: 0.0675
Epoch 2/100
4471/4471 [==============================] - 88s 20ms/step - loss: 6.1379 - accuracy: 0.1224
Epoch 3/100
4471/4471 [==============================] - 89s 20ms/step - loss: 5.6705 - accuracy: 0.1545
Epoch 4/100
4471/4471 [==============================] - 88s 20ms/step - loss: 5.3077 - accuracy: 0.1769
Epoch 5/100
4471/4471 [==============================] - 87s 20ms/step - loss: 5.0131 - accuracy: 0.1934
Epoch 6/100
4471/4471 [==============================] - 88s 20ms/step - loss: 4.7680 - accuracy: 0.2091
Epoch 7/100
4471/4471 [==============================] - 88s 20ms/step - loss: 4.5587 - accuracy: 0.2202
Epoch 8/100
4471/4471 [==============================] - 88s 20ms/step - loss: 4.3767 - accuracy: 0.2315
Epoch 9/100
4471/4471 [==============================] - 88s 20ms/step - loss: 4.2161 - accuracy: 0.2402
Epoch 10/100
4471/4471 [==============================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4471/4471 [==============================] - 86s 19ms/step - loss: 1.6667 - accuracy: 0.5965
Epoch 45/100
4471/4471 [==============================] - 86s 19ms/step - loss: 1.6404 - accuracy: 0.6015
Epoch 46/100
3679/4471 [=======================>......] - ETA: 15s - loss: 1.6022 - accuracy: 0.6110

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4471/4471 [==============================] - 86s 19ms/step - loss: 1.5903 - accuracy: 0.6141
Epoch 48/100
4471/4471 [==============================] - 86s 19ms/step - loss: 1.5697 - accuracy: 0.6174
Epoch 49/100
2978/4471 [==================>...........] - ETA: 28s - loss: 1.5070 - accuracy: 0.6308

...

4471/4471 [==============================] - 86s 19ms/step - loss: 1.5262 - accuracy: 0.6271
Epoch 51/100
4471/4471 [==============================] - 86s 19ms/step - loss: 1.5057 - accuracy: 0.6310
Epoch 52/100
2596/4471 [================>.............] - ETA: 35s - loss: 1.4361 - accuracy: 0.6468

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4471/4471 [==============================] - 86s 19ms/step - loss: 1.4718 - accuracy: 0.6381
Epoch 54/100
4471/4471 [==============================] - 86s 19ms/step - loss: 1.4568 - accuracy: 0.6433
Epoch 55/100
2541/4471 [================>.............] - ETA: 36s - loss: 1.4019 - accuracy: 0.6547

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4471/4471 [==============================] - 86s 19ms/step - loss: 1.4216 - accuracy: 0.6512
Epoch 57/100
4471/4471 [==============================] - 86s 19ms/step - loss: 1.4153 - accuracy: 0.6506
Epoch 58/100
4471/4471 [==============================] - 88s 20ms/step - loss: 1.3963 - accuracy: 0.6555
Epoch 59/100
4471/4471 [==============================] - 87s 19ms/step - loss: 1.3879 - accuracy: 0.6589
Epoch 60/100
4471/4471 [==============================] - 86s 19ms/step - loss: 1.3771 - accuracy: 0.6618
Epoch 61/100
4471/4471 [==============================] - 87s 19ms/step - loss: 1.3665 - accuracy: 0.6626
Epoch 62/100
4471/4471 [==============================] - 87s 19ms/step - loss: 1.3503 - accuracy: 0.6676
Epoch 63/100
4471/4471 [==============================] - 87s 19ms/step - loss: 1.3426 - accuracy: 0.6682
Epoch 64/100
4471/4471 [==============================] - 86s 19ms/step - loss: 1.3297 - accuracy: 0.6728
Epoch 65/100
4471/4471 [==============================] - 8

In [52]:
def generate_text(seed_text, num_words):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=input_size, padding='pre')
        predicted = model.predict(token_list)
        predicted_word_index = np.argmax(predicted)
        predicted_word = tokenizer.index_word[predicted_word_index]
        seed_text += " " + predicted_word
    return seed_text

In [37]:
# Save the model
model.save("clickbait_generator.h5")

In [56]:
# Example usage
seed_text = "programming finance"
num_words = 10
generated_text = generate_text(seed_text, num_words)
print(generated_text)

1/1 [==============================] - 0s 26ms/step
programming finance yourself the disabled dwarf musical of last gomez and more
